In [1]:
var tf = require('@tensorflow/tfjs-node');
var { plot, Plot } = require('nodeplotlib');

In [2]:
var file_path = "./dataset/gold_price.json";
var data = JSON.parse(fs.readFileSync(file_path));
data;

[
  { usd_price: '95.735', date: '2018-12-31T00:00:00.000Z' },
  { usd_price: '95.965', date: '2018-12-28T00:00:00.000Z' },
  { usd_price: '96.001', date: '2018-12-27T00:00:00.000Z' },
  { usd_price: '96.568', date: '2018-12-26T00:00:00.000Z' },
  { usd_price: '96.007', date: '2018-12-24T00:00:00.000Z' },
  { usd_price: '96.455', date: '2018-12-21T00:00:00.000Z' },
  { usd_price: '95.728', date: '2018-12-20T00:00:00.000Z' },
  { usd_price: '96.484', date: '2018-12-19T00:00:00.000Z' },
  { usd_price: '96.57', date: '2018-12-18T00:00:00.000Z' },
  { usd_price: '97.175', date: '2018-12-17T00:00:00.000Z' },
  { usd_price: '97.43', date: '2018-12-14T00:00:00.000Z' },
  { usd_price: '97.056', date: '2018-12-13T00:00:00.000Z' },
  { usd_price: '97.018', date: '2018-12-12T00:00:00.000Z' },
  { usd_price: '97.358', date: '2018-12-11T00:00:00.000Z' },
  { usd_price: '97.183', date: '2018-12-10T00:00:00.000Z' },
  { usd_price: '96.476', date: '2018-12-07T00:00:00.000Z' },
  { usd_price: '96.764',

In [3]:
function round_two_decimal(num) {
    var with2Decimals = num.toString().match(/^-?\d+(?:\.\d{0,2})?/)[0];
    return with2Decimals;
}

var get_gold_price = (filter_year) => {
    var filtered_data = data.filter(x => {
        var year = x['date'].substring(0,4);
        return year === filter_year;
    });
    var sorted_data = filtered_data.map(x => {
        return {
            'usd_price': round_two_decimal(x['usd_price']),
            'date': new Date(x['date']),
        };
    });
    var sort_asc = sorted_data.sort((a,b)=>a['date'].getTime()-b['date'].getTime());
    let result = sort_asc.map(x => {
       var usd_price = x['usd_price'];
       var date = x['date'].toISOString().substring(0,10);
       return {
           'usd_price': usd_price,
           'date' : date,
       };
    });
    return result;
};

In [4]:
//var gold price 2017
var gold_price_2017 = get_gold_price('2017');
//var gold price 2018
var gold_price_2018 = get_gold_price('2018');

In [5]:
// original chart - 2017 gold price
x2017 = [];
y2017 = [];

gold_price_2017.forEach(e => {
   let price = e['usd_price'];
   let date = e['date']; 
    x2017.push(date);
    y2017.push(price);
});

var third_plot_metadata= { x: x2017 , y: y2017, type: 'plot' , name: 'actual gold price',line: {color: "#128bcc" } }
//plot([third_plot_metadata]);

## RNN code begins here

In [6]:
function addTimeSeriesToSMA(data) {
    result = [];
    for (let index = 0; index < data.length;index++) {
        var x = data[index];
        var set = x['set'];
        var start_date = set[0]['date'];
        var end_date = set[set.length - 1]['date']; // date will be based on the start of the week
        var date = end_date;
        
        index += 1;
        let sma = {
            'set' : set,
            'avg' : x['avg'],
            'date': date, 
        }
        result.push(sma);
    }
    return result;
}

// SMA - smooth smoothing average over the day period
function computeSMA(data,window_size) {
let r_avgs=[];
let avg_prev=0;
  for(let i=0;i<data.length-window_size;i++){
     let curr_avg=0.00;
     let t=i+window_size;
     for(let j=i;j<t && j<=data.length;j++){
      curr_avg+=data[j]['usd_price']/window_size;
      }
     let sma={
     set:data.slice(i,i+window_size),
     avg:curr_avg,    
  }
  r_avgs.push(sma);
  avg_prev=curr_avg;
  }
console.log(r_avgs.length);    
r_avgs = addTimeSeriesToSMA(r_avgs);
    
return r_avgs;
}



In [7]:
// get the moving average for year 2017
var window_size_30 = 30;
var computeSMA2017_30 = computeSMA(gold_price_2017,window_size_30);

// get the moving average for year 2017
var window_size_50 = 50;
var computeSMA2017_50 = computeSMA(gold_price_2017,window_size_50);

218
198


In [8]:
// original chart - 2017 gold price (30 SMA)
x2017SMA_30 = [];
y2017SMA_30 = [];

computeSMA2017_30.forEach(e => {
   let price = e['avg'];
   let date = e['date']; 
    x2017SMA_30.push(date);
    y2017SMA_30.push(price);
});


var sma2017graph_30sma = { x: x2017SMA_30 , y: y2017SMA_30, type: 'plot' , name: '(30) SMA', line: {color: "#e3a909" } }

In [9]:
// original chart - 2017 gold price
x2017SMA_50 = [];
y2017SMA_50 = [];

computeSMA2017_50.forEach(e => {
   let price = e['avg'];
   let date = e['date']; 
    x2017SMA_50.push(date);
    y2017SMA_50.push(price);
});


var sma2017graph_50sma = { x: x2017SMA_50 , y: y2017SMA_50, type: 'plot' , name: '(50) SMA', line: {color: "#a16813" } }

In [10]:
plot([third_plot_metadata, sma2017graph_30sma, sma2017graph_50sma]);

[Nest] 15950  - 04/30/2022, 7:59:52 PM     LOG [NestFactory] Starting Nest application...
[Nest] 15950  - 04/30/2022, 7:59:52 PM     LOG [InstanceLoader] ServerModule dependencies initialized +23ms
[Nest] 15950  - 04/30/2022, 7:59:52 PM     LOG [InstanceLoader] ServeStaticModule dependencies initialized +0ms
[Nest] 15950  - 04/30/2022, 7:59:52 PM     LOG [WebSocketsController] PlotsGateway subscribed to the "readplots" message +42ms
[Nest] 15950  - 04/30/2022, 7:59:52 PM     LOG [NestApplication] Nest application successfully started +4ms
Server running at http://localhost:62517
client connected
client disconnected
Server shutting down


In [18]:
// Define input, which has a size of 5 (not including batch dimension).
var input = tf.input({shape: [5]});



// First dense layer uses relu activation.
var denseLayer1 = tf.layers.dense({units: 10, activation: 'relu'});
// Second dense layer uses softmax activation.
var denseLayer2 = tf.layers.dense({units: 4, activation: 'softmax'});

// Obtain the output symbolic tensor by applying the layers on the input.
var output = denseLayer2.apply(denseLayer1.apply(input));

// Create the model based on the inputs.
var model = tf.model({inputs: input, outputs: output});

// The model can be used for training, evaluation and prediction.
// For example, the following line runs prediction with the model on
// some fake data.
model.predict(tf.ones([2, 5])).print();

Tensor
    [[0.3292046, 0.205144, 0.1675977, 0.2980537],
     [0.3292046, 0.205144, 0.1675977, 0.2980537]]


In [ ]:
// hyper parameters

num_epochs = 100 // no of batches to train against
total_series_length = 50000
state_size = 4 // how neurons in the hidden layer
num_classess = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length

In [ ]:
async function trainModel(input, outputs, trainingsize,window_size, n_epochs, learning_rate, n_layers, callback) {
    // hyper parameters
    const input_layer_shape = window_size;
    const input_layer_neurons = 100;
    
    const rnn_input_layer_features = 10;
    const rnn_input_layer_timesteps = input_layer_neurons / rnn_input_layer_features; // 10
    
    const rnn_input_shape = [ rnn_input_layer_features , rnn_input_layer_timesteps ];
    const rnn_output_neurons = 20;
    
    const rnn_batch_size = window_size;
    
    const output_layer_shape = rnn_output_neurons;
    const output_layer_neurons = 1;
}